# tl;dr

This example runs models locally, but outputs the results of a hyperparameter grid search to a managed `Experiment`. 

See [here](https://console.cloud.google.com/vertex-ai/locations/us-central1/experiments/search-v1-20211001233901/metrics?project=dataorg-hackweek-2021) for an example. 

## Installation

Install the latest version of Vertex SDK for Python.

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

Extract project ID

In [2]:
import os

PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  dataorg-hackweek-2021


Basically get a timestamp and authentice (which we shouldn't because this is AI notebook)

In [3]:
from datetime import datetime
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

Bucket to save things off (assummed to exist)
see `bq_vertexai_train_deploy_search_v1.ipynb` on how to create

In [4]:
BUCKET_NAME = "gs://tmp-hackweek2021-model-training"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

List the items in the bucket

In [5]:
! gsutil ls -al $BUCKET_NAME

      4139  2021-09-30T17:05:30Z  gs://tmp-hackweek2021-model-training/aiplatform-2021-09-30-17:05:30.399-aiplatform_custom_trainer_script-0.1.tar.gz#1633021530511685  metageneration=1
      4539  2021-09-30T23:00:27Z  gs://tmp-hackweek2021-model-training/aiplatform-2021-09-30-23:00:27.039-aiplatform_custom_trainer_script-0.1.tar.gz#1633042827164033  metageneration=1
      4133  2021-09-30T23:10:45Z  gs://tmp-hackweek2021-model-training/aiplatform-2021-09-30-23:10:44.922-aiplatform_custom_trainer_script-0.1.tar.gz#1633043445045222  metageneration=1
      4533  2021-09-30T23:15:59Z  gs://tmp-hackweek2021-model-training/aiplatform-2021-09-30-23:15:59.246-aiplatform_custom_trainer_script-0.1.tar.gz#1633043759351736  metageneration=1
      4361  2021-09-30T23:26:56Z  gs://tmp-hackweek2021-model-training/aiplatform-2021-09-30-23:26:56.424-aiplatform_custom_trainer_script-0.1.tar.gz#1633044416509545  metageneration=1
      2067  2021-09-30T23:42:57Z  gs://tmp-hackweek2021-model-training/aipl

## AI Platform Config

Initialize Vertex AI and the experiment.

In [6]:
import os
import sys

from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip

EXPERIMENT_NAME = "search-v1-" + TIMESTAMP

aiplatform.init(project=PROJECT_ID, location=REGION, 
                staging_bucket=BUCKET_NAME,
                experiment=EXPERIMENT_NAME)

INFO:root:Resource search-v1-20211001233901 not found.
INFO:root:Creating Resource search-v1-20211001233901


## Model Train Config

## Custom Python Training Code

This utilize train/test/validation splits the `bq_vertexai_train_deploy_search_v1.ipynb` example. 

In [24]:
import argparse
import tensorflow as tf
import numpy as np
import os

import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

from sklearn.preprocessing import LabelEncoder

from google.cloud import bigquery
from google.cloud import storage

# Utilizes previously created tables from the other example
training_data_uri = 'bq://dataorg-hackweek-2021.dataset_3809367985592729600_tables_2021_09_30T16_00_27_766Z.training'
validation_data_uri = 'bq://dataorg-hackweek-2021.dataset_3809367985592729600_tables_2021_09_30T16_00_27_766Z.validation'
test_data_uri = 'bq://dataorg-hackweek-2021.dataset_3809367985592729600_tables_2021_09_30T16_00_27_766Z.test'


# Set up BigQuery clients
bqclient = bigquery.Client()

# Download a table
def download_table(bq_table_uri: str):
    # Remove bq:// prefix if present
    prefix = "bq://"
    if bq_table_uri.startswith(prefix):
        bq_table_uri = bq_table_uri[len(prefix):]

    table = bigquery.TableReference.from_string(bq_table_uri)
    rows = bqclient.list_rows(
        table,
    )
    return rows.to_dataframe(create_bqstorage_client=False)


df_train = download_table(training_data_uri)
df_validation = download_table(validation_data_uri)
df_test = download_table(test_data_uri)

df_train = df_train[~df_train.title.isnull()]
df_validation = df_validation[~df_validation.title.isnull()]
df_test = df_test[~df_test.title.isnull()]

# create the model
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_len,
)

train_texts = df_train["title"].to_numpy()
vectorize_layer.adapt(train_texts)

# format the labels
encoder = LabelEncoder()
encoder.fit(sorted(df_train["selected"].unique()))
labels = encoder.transform(df_train['selected'].tolist())
val_labels = encoder.transform(df_validation['selected'].tolist())
test_labels = encoder.transform(df_test['selected'].tolist())

def train(X_train, y_train, X_validation, y_validation, epochs, nunits):
    model = Sequential()

    model.add(Input(shape=(1,), dtype="string"))
    model.add(vectorize_layer)
    model.add(Embedding(max_tokens + 1, 128))
    model.add(LSTM(nunits))
    model.add(Dense(nunits, activation="relu"))
    model.add(Dense(df_train["labels"].unique().size, activation="sigmoid"))
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    # train the model
    history = model.fit(X_train, y_train, epochs=epochs, 
                        validation_data=(X_validation, y_validation))
    return(model, history)

In [23]:
parameters = [
    {"num_units": 32, "epochs": 3},
    {"num_units": 32, "epochs": 10},
    {"num_units": 32, "epochs": 20},
    {"num_units": 64, "epochs": 3},
    {"num_units": 64, "epochs": 10},
    {"num_units": 64, "epochs": 20}
]

for i, params in enumerate(parameters):
    aiplatform.start_run(run=f"search-v1-local-run-{i}")
    aiplatform.log_params(params)
    model, history = train(
        df_train["title"].to_numpy(), 
        labels, 
        df_validation["title"].to_numpy(), 
        val_labels,               
        epochs=params["epochs"],
        nunits=params["num_units"]
    )
    aiplatform.log_metrics(
        {metric: values[-1] for metric, values in history.history.items()}
    )

    loss, accuracy = model.evaluate(df_test['title'].to_numpy(), test_labels, verbose=2)
    aiplatform.log_metrics({"eval_loss": loss, "eval_accuracy": mae, "eval_mse": mse})

Epoch 1/3
39/39 [==============================] - 5s 139ms/step - loss: 4.7242 - accuracy: 0.7693 - val_loss: 4.3367 - val_accuracy: 0.8630
Epoch 2/3
39/39 [==============================] - 5s 122ms/step - loss: 4.1472 - accuracy: 0.8645 - val_loss: 4.0492 - val_accuracy: 0.8630
Epoch 3/3
39/39 [==============================] - 5s 124ms/step - loss: 4.0380 - accuracy: 0.8645 - val_loss: 4.0326 - val_accuracy: 0.8630
5/5 - 0s - loss: 4.0327 - accuracy: 0.7895


ValueError: not enough values to unpack (expected 3, got 2)

In [25]:
model.evaluate(df_test['title'].to_numpy(), test_labels, verbose=2)

5/5 - 0s - loss: 4.0327 - accuracy: 0.7895


[4.032702922821045, 0.7894737124443054]